### Creates .json parameter files from validated catalog combinations

In [ ]:
import pandas as pd
from tqdm import tqdm
import json
from cmip6_downscaling.methods.common.containers import RunParameters

In [ ]:
valid_df = pd.read_csv('valid_catalog.csv')

template = {
    "bcsd": {
        "method": "",
        "obs": "ERA5",
        "model": "",
        "member": "",
        "grid_label": "gn",
        "table_id": "day",
        "scenario": "",
        "features": "",
        "variable": "",
        "train_dates": ["1981", "2010"],
        "predict_dates": "",
        "latmin": "-90",
        "latmax": "90",
        "lonmin": "-180",
        "lonmax": "180",
        "bias_correction_method": "",
        "bias_correction_kwargs": {},
        "model_type": "",
        "model_params": {},
    },
    "gard": {
        "method": "",
        "obs": "ERA5",
        "model": "",
        "member": "",
        "grid_label": "gn",
        "table_id": "day",
        "scenario": "",
        "features": "",
        "variable": "",
        "train_dates": ["1981", "2010"],
        "predict_dates": "",
        "latmin": "-90",
        "latmax": "90",
        "lonmin": "-180",
        "lonmax": "180",
        "bias_correction_method": "quantile_mapper",
        "bias_correction_kwargs": {},
        "model_type": "PureRegression",
        "model_params": {},
    },
}

In [ ]:
gcms = ['MIROC6', 'CanESM5', 'BCC-CSM2-MR', 'MPI-ESM1-2-HR', 'MRI-ESM2-0']
member = 'r1i1p1f1'
variables = ['tasmax', 'tasmin', 'pr']

In [ ]:
for method in ['bcsd', 'gard']:
    for gcm in gcms:
        for variable in variables:
            subset = valid_df[valid_df['source_id'] == gcm]

            subset = subset[subset['member_id'] == member]
            subset = subset[subset['variable_id_y'] == variable]
            print(len(subset))

            for index, row in tqdm(subset.iterrows()):
                filled_template = template[method].copy()
                filled_template["method"] = method
                filled_template["member"] = row["member_id"]
                filled_template["model"] = row["source_id"]
                filled_template["features"] = [row["variable_id_y"]]
                filled_template["variable"] = row["variable_id_y"]
                filled_template["scenario"] = row["experiment_id_y"]

                if 'ssp' in filled_template["scenario"]:
                    filled_template["predict_dates"] = ["2015", "2099"]
                else:
                    filled_template["predict_dates"] = ["1950", "2014"]
                if variable == 'pr':
                    filled_template['model_params'] = {'thresh': 0}
                    filled_template['bias_correction_kwargs'] = {"detrend": "False"}
                elif variable in ['tasmax', 'tasmin']:
                    filled_template['bias_correction_kwargs'] = {"detrend": "True"}
                filled_template['train_dates'] = filled_template.pop('train_dates')
                filled_template['predict_dates'] = filled_template.pop('predict_dates')

                run_parameters = RunParameters(**filled_template)

                with open(f'{method}' + '/' + run_parameters.run_id + '.json', 'w') as f:
                    f.write(json.dumps(filled_template))